In [ ]:
library(EpiDISH)
library(readxl)
library(reticulate)
library(ggplot2)
library(tools)
library(reshape2)
library(RColorBrewer)
library(dplyr)
library(tidyr)

In [ ]:

use_python("C:/Users/17599/Documents/.virtualenvs/r-reticulate/Scripts/python.exe") 
source_python("vif_calculation.py")
source("correlation_functions.R")
source("cell_frac_cuz_change.r")
source("correlation_functions.R")

cpg_data <- load_data()
hannum_data <- cpg_data$hannum
arth_nor <- cpg_data$arth_nor
arth_dis <- cpg_data$arth_dis

dir.create("results", showWarnings = FALSE)
cpg_files <- list.files("E:/SynologyDrive/MXY/MXY/conboy_lab_codeSource/Rebuttal/clock_model_coef", pattern=".*\\.(csv|xlsx?)$", full.names=TRUE)
# estimate cell fractions through epidish
data(cent12CT450k.m)


In [ ]:
for (cpg_file in cpg_files) {
    clock_name <- file_path_sans_ext(basename(cpg_file))

    # read csv or excel file
    if (grepl("\\.csv$", cpg_file, ignore.case=TRUE)) {
        model_df <- read.csv(cpg_file)
    } else if (grepl("\\.xlsx?$", cpg_file, ignore.case=TRUE)) {
        model_df <- read_excel(cpg_file)
    } else {
        next 
    }

    model_cpgs <- detect_columns(model_df)
    model_cpgs <- model_cpgs[!grepl("intercept", model_cpgs, ignore.case=TRUE)]  # Remove 'Intercept' entries
    model_cpgs <- intersect(model_cpgs, colnames(hannum_data))

    # ==== compute correlations====
    correlations <- compute_correlation(hannum_data, arth_nor, arth_dis, model_cpgs)
    print(plot_correlation(correlations$hannum, paste(clock_name, "- Hannum Model CpGs vs Cell Fractions")))
    print(plot_correlation(correlations$arth_nor, paste(clock_name, "- Arth Nor Model CpGs vs Cell Fractions")))
    print(plot_correlation(correlations$arth_dis, paste(clock_name, "- Arth Dis Model CpGs vs Cell Fractions")))

    # ==== compute vif ======
    common_cpgs <- intersect(model_cpgs, colnames(hannum_data))
    hannum_selected <- hannum_data[, common_cpgs, drop=FALSE]


    hannum_cell_fractions <- epidish(beta.m = t(hannum_data), ref.m = cent12CT450k.m, method = "RPC")$estF

    # working code but used to plot the original vif and cell fraction corr plots
    hannum_combined <- cbind(hannum_selected, hannum_cell_fractions)
    
    vif_results <- vif_with_features(py$pd$DataFrame(hannum_combined))    # py for better manipulation
    vif_results <- as.data.frame(vif_results)
    colnames(vif_results) <- c("Feature", "VIF")
    vif_results <- vif_results[is.finite(vif_results$VIF), , drop=FALSE]
    vif_plot <- plot_vif_pie(vif_results, clock_name)
}

In [ ]:
output_dir <- "E:/SynologyDrive/MXY/MXY/conboy_lab_codeSource/Rebuttal/clock_vif_results"
dir.create(output_dir, showWarnings = FALSE, recursive = TRUE)

# quadrant scatterplots for Pearson correlation and assigned coefficient
common_epidish_cpgs <- intersect(rownames(cent12CT450k.m), rownames(t(hannum_data)))
beta_mat_full <- t(hannum_data)[common_epidish_cpgs, ]
ref_mat_full <- cent12CT450k.m[common_epidish_cpgs, ]
hannum_cell_fractions <- epidish(beta.m = beta_mat_full, ref.m = ref_mat_full, method = "RPC")$estF

# age and covariates 
age_col <- colnames(hannum_data)[tolower(colnames(hannum_data)) == "age"]
ages <- hannum_data[[age_col]]
covariates_full <- hannum_cell_fractions

# redundant but for separate plots
for (cpg_file in cpg_files) {
  clock_name <- tools::file_path_sans_ext(basename(cpg_file))

  model_df <- if (grepl("\\.csv$", cpg_file, ignore.case=TRUE)) {
    read.csv(cpg_file)
  } else if (grepl("\\.xlsx?$", cpg_file, ignore.case=TRUE)) {
    read_excel(cpg_file)
  } else {
    stop("Invalid file format.")
  }

  model_cpgs <- detect_columns(model_df)
  model_cpgs <- model_cpgs[!grepl("intercept", model_cpgs, ignore.case = TRUE)]
  model_cpgs <- intersect(model_cpgs, colnames(hannum_data))
  common_cpgs <- model_cpgs

  if (length(common_cpgs) < 2) {
    cat("Skipping", clock_name, "- too few common CpGs.\n")
    next
  }

  hannum_selected <- hannum_data[, common_cpgs, drop = FALSE]

  # UNADJUSTED
  univ_corr_unadj <- apply(hannum_selected, 2, function(col) {
    cor(col, ages, method = "pearson", use = "complete.obs")
  })

  # ADJUSTED
  univ_corr_adj <- numeric(length(common_cpgs))
  names(univ_corr_adj) <- common_cpgs

  for (cpg in common_cpgs) {
    df <- data.frame(
      age = ages,
      cpg_val = hannum_data[[cpg]],
      covariates_full
    )
    fit <- lm(age ~ cpg_val + ., data = df)
    univ_corr_adj[cpg] <- coef(fit)["cpg_val"]
  }

  # max-normalize adjusted values to -1~ 1 for visualization
  univ_corr_adj <- univ_corr_adj / max(abs(univ_corr_adj), na.rm = TRUE)

  # load model weights and normalize
  model_weights <- get_normalized_weights(model_df, common_cpgs, clock_name)
  if (is.null(model_weights)) {
    cat("Skipping", clock_name, "- could not extract weights.\n")
    next
  }

  # --- summary df: UNADJUSTED ---
  agreement_df_unadj <- data.frame(
    CpG = common_cpgs,
    Univ = univ_corr_unadj[common_cpgs],
    Model = as.vector(model_weights),
    Disagree = sign(univ_corr_unadj[common_cpgs]) != sign(model_weights)
  )
  agreement_df_unadj <- agreement_df_unadj[complete.cases(agreement_df_unadj), ]
  plot_feature_quadrant(
    agreement_df_unadj,
    clock_name = paste0(clock_name, "_unadjusted")
  )

  # --- summary df: ADJUSTED ---
  agreement_df_adj <- data.frame(
    CpG = common_cpgs,
    Univ = univ_corr_adj[common_cpgs],
    Model = as.vector(model_weights),
    Disagree = sign(univ_corr_adj[common_cpgs]) != sign(model_weights)
  )
  agreement_df_adj <- agreement_df_adj[complete.cases(agreement_df_adj), ]
  plot_feature_quadrant(
    agreement_df_adj,
    clock_name = paste0(clock_name, "_adjusted")
  )
}


In [ ]:
# VIF differences when adding fraction of one cell type
hannum_cell_fractions <- epidish(beta.m = t(hannum_data), ref.m = cent12CT450k.m, method = "RPC")$estF
initial_vif <- vif_with_features(py$pd$DataFrame(hannum_selected))

# set feature column to row names for standard format
if ("Feature" %in% colnames(initial_vif)) {
    rownames(initial_vif) <- initial_vif$Feature
    initial_vif <- initial_vif %>% select(-Feature) 
}

#baseline vif
vif_increase_df <- initial_vif["VIF", drop = FALSE]  
colnames(vif_increase_df) <- "Baseline_VIF"  

for (cell_type in colnames(hannum_cell_fractions)) {
    hannum_with_cell <- cbind(hannum_selected, hannum_cell_fractions[, cell_type, drop=FALSE])
    vif_results <- vif_with_features(py$pd$DataFrame(hannum_with_cell))
    if ("Feature" %in% colnames(vif_results)) {
        rownames(vif_results) <- vif_results$Feature
        vif_results <- vif_results %>% select(-Feature)
    }
    common_features <- intersect(rownames(vif_increase_df), rownames(vif_results))
    # vif when a cell type is added - baseline vif
    vif_increase_df[common_features, cell_type] <- vif_results[common_features, "VIF"] - initial_vif[common_features, "VIF"]
} 

vif_increase_df <- vif_increase_df[rownames(vif_increase_df) != "Intercept", ]
vif_long <- melt(vif_increase_df, variable.name = "CellType", value.name = "VIF_Change")

vif_long <- vif_long %>%
    mutate(VIF_Category = factor(case_when(
        abs(VIF_Change) < 0.025 ~ "Small Change (<0.025)",
        abs(VIF_Change) >= 0.025 & abs(VIF_Change) < 0.05 ~ "Moderate Change (0.025-0.05)",
        abs(VIF_Change) >= 0.05 & abs(VIF_Change) < 0.1 ~ "Large Change (0.05-0.1)",
        abs(VIF_Change) >= 0.1 ~ "Very Large Change (>0.1)"
    ), levels = c(
        "Small Change (<0.025)", 
        "Moderate Change (0.025-0.05)", 
        "Large Change (0.05-0.1)", 
        "Very Large Change (>0.1)"
    )))


vif_pie_data <- vif_long %>%
    group_by(CellType, VIF_Category) %>%
    summarise(Count = n()) %>%
    ungroup()

category_colors <- c(
    "Small Change (<0.025)" = "#1f77b4",
    "Moderate Change (0.025-0.05)" = "#ff7f0e",
    "Large Change (0.05-0.1)" = "#2ca02c",
    "Very Large Change (>0.1)" = "#d62728"
)

# pie chart faceted by cell type
pie_chart <- ggplot(vif_pie_data, aes(x = "", y = Count, fill = VIF_Category)) +
    geom_bar(stat = "identity", width = 1) + 
    coord_polar("y", start = 0) +  
    facet_wrap(~ CellType, ncol = 4) +  
    scale_fill_manual(values = category_colors, breaks = names(category_colors)) +  # Ensures correct order
    labs(title = paste("Distribution of VIF Change Categories for", clock_name),
         fill = "VIF Change Category") +
    theme_minimal() +
    theme(axis.text.x = element_blank(), 
          axis.ticks = element_blank(),  
          panel.grid = element_blank())  # better visual

file_path <- file.path(output_dir, paste0(clock_name, "_cell_frac_vif.png"))
ggsave(filename = file_path, plot = pie_chart, width = 12, height = 8, dpi = 300)
print(paste("Saved file:", file_path))
